In [6]:
%time

Wall time: 0 ns


In [7]:
import pandas as pd

In [4]:
# Reading all the files
raw_path = 'C:/Users/siyuf/Documents/00. MiTB/H&M/H&M/'
articles = pd.read_csv(raw_path + 'articles.csv')
customer = pd.read_csv(raw_path + 'customers.csv')
transaction = pd.read_csv(raw_path + 'transactions_train.csv')


In [14]:
def data_overview(df, corr=False, label_name=None, sort_by='qtd_null', thresh_percent_null=0, thresh_corr_label=0):
    """
    Etapas:
        1. levantamento de atributos com dados nulos no conjunto
        2. análise do tipo primitivo de cada atributo
        3. análise da quantidade de entradas em caso de atributos categóricos
        4. extração da correlação pearson com o target para cada atributo
        5. aplicação de regras definidas nos argumentos
        6. retorno do dataset de overview criado

    Argumentos:
        df -- DataFrame a ser analisado [pandas.DataFrame]
        label_name -- nome da variável target [string]
        sort_by -- coluna de ordenação do dataset de overview [string - default: 'qtd_null']
        thresh_percent_null -- filtro de dados nulos [int - default: 0]
        threh_corr_label -- filtro de correlação com o target [int - default: 0]

    Retorno
        df_overview -- dataet consolidado contendo análise das colunas [pandas.DataFrame]
    """

    # Criando DataFrame com informações de dados nulos
    df_null = pd.DataFrame(df.isnull().sum()).reset_index()
    df_null.columns = ['feature', 'qtd_null']
    df_null['percent_null'] = df_null['qtd_null'] / len(df)

    # Retornando tipo primitivo e qtd de entradas para os categóricos
    df_null['dtype'] = df_null['feature'].apply(lambda x: df[x].dtype)
    df_null['qtd_cat'] = [len(df[col].value_counts()) if df[col].dtype == 'object' else 0 for col in
                          df_null['feature'].values]

    if corr:
        # Extraindo informação de correlação com o target
        label_corr = pd.DataFrame(df.corr()[label_name])
        label_corr = label_corr.reset_index()
        label_corr.columns = ['feature', 'target_pearson_corr']

        # Unindo informações
        df_null_overview = df_null.merge(label_corr, how='left', on='feature')
        df_null_overview.query('target_pearson_corr > @thresh_corr_label')
    else:
        df_null_overview = df_null

    # Filtrando dados nulos de acordo com limiares
    df_null_overview.query('percent_null > @thresh_percent_null')

    # Ordenando DataFrame
    df_null_overview = df_null_overview.sort_values(by=sort_by, ascending=False)
    df_null_overview = df_null_overview.reset_index(drop=True)

    return df_null_overview

In [8]:
# Collections for each dataset
datasets = [articles,customer, transaction]
names = ['articles', 'customer', 'transaction']

# Creating a DataFrame with useful information about all datasets
data_info = pd.DataFrame({})
data_info['dataset'] = names
data_info['n_rows'] = [df.shape[0] for df in datasets]
data_info['n_cols'] = [df.shape[1] for df in datasets]
data_info['null_amount'] = [df.isnull().sum().sum() for df in datasets]
data_info['qty_null_columns'] = [len([col for col, null in df.isnull().sum().items() if null > 0]) for df in datasets]
data_info['null_columns'] = [', '.join([col for col, null in df.isnull().sum().items() if null > 0]) for df in datasets]

data_info.style.background_gradient()

,dataset,n_rows,n_cols,null_amount,qty_null_columns,null_columns
0,articles,105542,25,416,1,detail_desc
1,customer,1371980,7,1840558,5,"FN, Active, club_member_status, fashion_news_frequency, age"
2,transaction,31788324,5,0,0,


In [15]:
df_overview = pd.DataFrame()
name_col = []
for name, df in zip(names, datasets):
    name_col += [name] * df.shape[1]
    df_overview = df_overview.append(data_overview(df))
    df_overview['dataset_name'] = name_col

df_overview = df_overview.loc[:, ['dataset_name', 'feature', 'qtd_null', 'percent_null', 'dtype', 'qtd_cat']]
df_overview

,dataset_name,feature,qtd_null,percent_null,dtype,qtd_cat
0,articles,detail_desc,416,0.003942,object,43404
1,articles,perceived_colour_master_name,0,0.000000,object,20
2,articles,garment_group_name,0,0.000000,object,21
3,articles,garment_group_no,0,0.000000,int64,0
4,articles,section_name,0,0.000000,object,56
5,articles,section_no,0,0.000000,int64,0
6,articles,index_group_name,0,0.000000,object,5
7,articles,index_group_no,0,0.000000,int64,0
8,articles,index_name,0,0.000000,object,10
9,articles,index_code,0,0.000000,object,10


In [16]:
df_overview.to_excel('saved_file.xlsx')